In [ ]:
import numpy as np
import geopandas
import matplotlib
import matplotlib.ticker as plticker
import pandas as pd
import json
import contextily
import matplotlib.pyplot as plt
import geoplot
import seaborn as sns
import datetime
import csv

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from shapely.geometry import Polygon, Point

### Data inspection & cleaning

In [ ]:
#for desktop
#df = pd.read_csv(r'C:\Users\Ray\Desktop\TweetsAmsterdamWithLanguage.csv', encoding = 'latin1', sep = ',')

In [ ]:
#for laptop
df = pd.read_csv(r'C:\Users\raymo\Downloads\AmsterdamWithLanguage.csv', encoding = 'latin1', sep = ',')

In [ ]:
df.info()

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

### Data transformation and descriptive statistics

In [ ]:
df['created_at'] = pd.to_datetime(df['created_at'], format='%a %b %d %H:%M:%S +0000 %Y')

In [ ]:
### transform from GMT to UTC by adding one hour
df['created_at'] = df['created_at'] + datetime.timedelta(hours=1)

In [ ]:
#remove 2011 and 2019 from dataframe
start_date = '01-01-2012'
end_date = '01-01-2019'
mask = (df['created_at'] >= start_date) & (df['created_at'] < end_date)
df = df.loc[mask]

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
#Delete bots

In [ ]:
df['user_id'].value_counts()

In [ ]:
df[df.user_name == 'Sol Tempore'].head()

In [ ]:
### Manually inspect user ids for bots
df[df.user_id ==215292728].head(10)


In [ ]:
#delete user_ids which are bots
df = df[(df.user_id !=272419626) & (df.user_id !=186899860) & (df.user_id !=550261599) & (df.user_id !=291158798) 
        &(df.user_id !=19140633) &(df.user_id !=278708054) &(df.user_id !=3375579785) &(df.user_id !=208056970) & 
        (df.user_id !=88039815) & (df.user_id !=168489388 ) & (df.user_id !=191092262) & (df.user_id !=215292728)]


In [ ]:
df

In [ ]:
#Drop duplicates

In [ ]:
df['id'].value_counts()

In [ ]:
df = df.drop_duplicates(subset='id', keep="first")

In [ ]:
df.count()

In [ ]:
df.info()

In [ ]:
#Continue with dates

In [ ]:
def get_day_of_week(day):
    weekdays = {
        0: 'Monday',
        1: 'Tuesday',
        2: 'Wednesday',
        3: 'Thursday',
        4: 'Friday',
        5: 'Saturday',
        6: 'Sunday'
    }
    return weekdays[day]

def get_season(month):
    months = {
        1: 'Winter',
        2: 'Winter',
        3: 'Spring',
        4: 'Spring',
        5: 'Spring',
        6: 'Summer',
        7: 'Summer',
        8: 'Summer',
        9: 'Autumn',
        10: 'Autumn',
        11: 'Autumn',
        12: 'Winter',
    }
    
    return months[month]

df['Year'] = df.created_at.dt.year
df['Month'] = df.created_at.dt.month
df['Day'] = df.created_at.dt.day
df['just_date'] = df['created_at'].dt.date
df['Hour'] = df.created_at.dt.hour
df['DayOfWeek_nr'] = df.created_at.dt.dayofweek
df['DayOfWeek'] = df.created_at.dt.dayofweek.apply(lambda x: get_day_of_week(x))
df['YearMonth'] = pd.to_datetime(df['created_at']).dt.to_period('M')
df['Season'] = df.Month.apply(get_season)

In [ ]:
#start_date = '09-01-2013'
#end_date = '09-08-2013'
#mask = (df['created_at'] > start_date) & (df['created_at'] <= end_date)
#df2 = df.loc[mask]
#df2.id = df.id.astype(str)
#df2.id_str = df.id_str.astype(str)
#df2.to_csv(r'C:\Users\raymo\Downloads\AmsterdamSeptember2013-09-01.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
#df2.info()


In [ ]:
df[
    (~df.language.isna())
    &
    (df.Year == 2013)
    &
    (df.Month > 1)
    &
    (df.Month < 4)
].created_at.dt.date.value_counts().reset_index().sort_values(by='index')
#highlights that language data is missin between 08-02-2013 and 26-03-2013

In [ ]:
#only to set style for matplotlib plots
sns.scatterplot(x='Hour',y='created_at',data=df)
sns.set(style="darkgrid")
#plt.show()

In [ ]:
df.groupby('Hour').agg({'id':'count'}).rename(columns={'id':'Tweets per hour'}).plot(title = 'Distribution of tweets per hour', kind = 'bar', color = 'royalblue', legend = None)
plt.xlabel('Hour')
plt.ylabel('Number of Tweets')
plt.tight_layout()
plt.savefig('Distribution of tweets per hour')
plt.show()


In [ ]:
df

In [ ]:
df.info()

In [ ]:
#df = df.reset_index()
#df = df.set_index(['created_at'])
df['just_date'] = pd.to_datetime(df['just_date'])
df.groupby(pd.Grouper(key='just_date', freq='W')).agg({'user_id' : 'count'}).plot(figsize = (20, 10), legend = None)
#df = df.reset_index()

In [ ]:
df.groupby('Year').agg({'id':'count'}).rename(columns={'id':'Tweets per year'}).plot(title = 'Distribution of tweets per year', kind = 'bar', color = 'royalblue', legend = None)
plt.ylabel('Number of Tweets')
plt.tight_layout()
plt.savefig('Distribution of tweets per year')
plt.show()

In [ ]:
ax = df.groupby('DayOfWeek_nr').agg({'id':'count'}).plot(title = 'Distribution of tweets per day of the week', kind = 'bar', color = 'royalblue', legend = None)
plt.xlabel('Weekday')
plt.ylabel('Number of Tweets')
ax.set_xticklabels(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
plt.tight_layout()
plt.savefig('Distribution of tweets per day of the week')
plt.show()

In [ ]:
ax = df.groupby('Month').agg({'id':'count'}).plot(title = 'Distribution of tweets per month', kind = 'bar', color = 'royalblue', legend = None)
plt.xlabel('Month')
plt.ylabel('Number of Tweets')
ax.set_xticklabels(['January','February','March','April','May','June','July', 'August', 'September', 'October', 'November', 'December'])
plt.tight_layout()
plt.savefig('Distribution of tweets per month')
plt.show()

In [ ]:
ax = df.groupby('Season').agg({'id':'count'}).plot(title = 'Distribution of tweets per season', kind = 'bar', color = 'royalblue', legend = None)
plt.xlabel('Season')
plt.ylabel('Number of Tweets')
plt.tight_layout()
plt.savefig('Distribution of tweets per Season')
plt.show()

### Distinguish tourists from residents

In [ ]:
user_date_ranges_df = df.groupby('user_id').agg({
    'created_at': ['min','max'],
    'tweet_text': ['count']
}).reset_index()

# time difference from first tweet to last tweet
user_date_ranges_df['diff'] = user_date_ranges_df['created_at']['max'] - user_date_ranges_df['created_at']['min']

user_date_ranges_df.columns = ['user_id','min','max','count','diff']
user_date_ranges_df = user_date_ranges_df.drop(columns=['min','max'])

# average number of tweets per day
user_date_ranges_df['freq'] = user_date_ranges_df['count'] / (user_date_ranges_df['diff'].dt.days)
user_date_ranges_df['freq'].replace(np.inf, 0, inplace=True)

In [ ]:
df_merged = df.merge(user_date_ranges_df, how='left', on='user_id')

In [ ]:
df_merged.sample(5)

In [ ]:
df_merged['is_tourist'] = (df_merged['diff'] < pd.Timedelta('14 days'))

In [ ]:
df_tourist = df_merged[df_merged.is_tourist == True]
df_not_tourist = df_merged[df_merged.is_tourist == False]

In [ ]:
df_tourist.info()

In [ ]:
# Also create dataset with ENG only for sentiment analysis (from 2013 onwards since language was added in this period)
df_tourist_en = df_tourist[df_tourist.language == 'en']
start_date = '01-01-2013'
end_date = '01-01-2019'
mask = (df_tourist_en['created_at'] >= start_date) & (df_tourist_en['created_at'] < end_date)
df_tourist_en = df_tourist_en.loc[mask]
#df_tourist_en.to_csv(r'C:\Users\raymo\Downloads\df_tourist_en.csv', index=False, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)

### Sentiment Analysis

In [ ]:
%%capture
analyser = SentimentIntensityAnalyzer()
df_tourist_en['neg'], df_tourist_en['neu'], df_tourist_en['pos'], df_tourist_en['compound'] = zip(
    *df_tourist_en['tweet_text'].apply(
        lambda x: list(analyser.polarity_scores(x).values())
    )
)

In [ ]:
def get_sentiment_label(compound):
    if(compound >= 0.05):
        return 'pos'
    elif(compound <= -0.05):
        return 'neg'
    else:
        return 'neu'

df_tourist_en['polarity'] = df_tourist_en['compound'].apply(lambda x: get_sentiment_label(x))

In [ ]:
df_tourist_en.polarity.value_counts()

In [ ]:
df_tourist_en['polarity'].astype(str)

In [ ]:
data_perc = df_tourist_en.groupby(pd.Grouper(key='just_date', freq='W')).agg({'neg': 'mean', 'neu': 'mean', 'pos': 'mean'})*100
data_perc.columns = ['perc. of negative tweets', 'perc. of neutral tweets', 'perc. of positive tweets']
data_perc.divide(data_perc.sum(axis=1), axis=0)
ax = data_perc.plot(figsize = (10,5))
ax.set_ylim(0,100)
plt.xlabel('Years')
plt.ylabel('Percentage')
plt.tight_layout()
plt.savefig('Day of year sentiment patterns')


In [ ]:
df_tourist_en = df_tourist_en[(df_tourist_en.user_id !=2463424754)]
by_day_sentiment = df_tourist_en.groupby([pd.Grouper(key='created_at', freq='D'), 'polarity']) \
    .size().unstack('polarity')

In [ ]:
by_day_sentiment.nlargest(10, ['pos'])

In [ ]:
data_perc = df_tourist_en.groupby(pd.Grouper(key='just_date', freq='D')).agg({'neg': 'mean', 'neu': 'mean', 'pos': 'mean'})*100
data_perc.columns = ['neg', 'neu', 'pos']
data_perc.divide(data_perc.sum(axis=1), axis=0)
ax = data_perc.plot(figsize = (20,10))
ax.set_ylim(0,100)

#plt.stackplot(data_perc["neg"],  data_perc["neu"],  data_perc["pos"], labels=['neg','neu','pos'])

In [ ]:
df_tourist_en.reset_index(inplace=True)
df_tourist_en = df_tourist_en[(df_tourist_en.user_id !=2463424754)]
by_day_sentiment = df_tourist_en.groupby([pd.Grouper(key='created_at', freq='D'), 'polarity']) \
    .size().unstack('polarity')

In [ ]:
by_day_sentiment.plot(figsize=(20, 10))
plt.xlabel('Years')
plt.ylabel('Polarity counts')
plt.tight_layout()
plt.savefig('Initial sentiment distribution')
plt.show()

In [ ]:
by_day_sentiment.nlargest(10, ['pos'])

In [ ]:
#Check messages on certain days
df_tourist_en = df_tourist_en.set_index(['created_at'])
df_tourist_en.loc['2014-02-08']


In [ ]:
#reset index
df_tourist_en.reset_index(inplace = True)

In [ ]:
#exclude neutral 
#df_tourist_en = df_tourist_en.loc[df_tourist_en['polarity'] != 'neu']

In [ ]:
df_tourist_en

In [ ]:
#Exclude hastag #canyoumakeit18
df_tourist_en = df_tourist_en[~df_tourist_en.tweet_text.str.contains('#canyoumakeit18')]

In [ ]:
#delete additional users who disrupt sentiment
df_tourist_en = df_tourist_en[(df_tourist_en.user_id !=2463424754)]

In [ ]:
data_perc = df_tourist_en.groupby(pd.Grouper(key='just_date', freq='D')).agg({'neg': 'mean', 'neu': 'mean', 'pos': 'mean'})*100
data_perc.columns = ['perc. of negative tweets', 'perc. of neutral tweets', 'perc. of positive tweets']
data_perc.divide(data_perc.sum(axis=1), axis=0)
ax = data_perc.plot(figsize = (20,10))
ax.set_ylim(0,100)
plt.xlabel('Years')
plt.ylabel('Percentage of tweets')
plt.tight_layout()
plt.savefig('Sentiment patterns')
plt.show()


In [ ]:
plt.figure(figsize=(10,10))
sns.set_style("whitegrid")
sns.set_palette("Blues")
sns.lineplot(x = 'Year', y = 'compound', data = df_tourist_en) 
plt.tight_layout()
plt.savefig('Sentiment per season')



In [ ]:
df_tourist_en.groupby('Year')['compound'].mean()

In [ ]:
df_tourist_en.groupby('Season')['compound'].mean()

In [ ]:
df_tourist_en.boxplot(column=['compound'], by=['DayOfWeek'], figsize=(15,10))


In [ ]:
with open('../Amsterdamlocations.json') as fname:
    amsterdam_locations = json.load(fname)
    
df_tourist_en['coordinate_point'] = df_tourist_en.apply(lambda row: Point(row['latitude'], row['longitude']), axis = 1)

df_tourist_en[['latitude','longitude','coordinate_point']].head()

In [ ]:
%%time
def in_location(locations, location):
    for place_name, area in locations.items():
        if(area['polygon'].contains(location)):
            return place_name
    return np.nan

# create Polygons of the Areas of Interest
for place, values in amsterdam_locations.items():
    amsterdam_locations[place]['polygon'] = Polygon(zip(values['x'], values['y']))

In [ ]:
# create categorical column where each tweet is mapped in 1 of 7 areas (or none)
df_tourist_en['POI'] = df_tourist_en.coordinate_point.apply(
    lambda location: in_location(
        amsterdam_locations,
        location
    )
)

In [ ]:
df_tourist_en.groupby(by=['POI','Year']).id.count()

In [ ]:
data = df_tourist_en.groupby(by=['POI', 'Year']).compound.mean().reset_index()
data.set_index('Year', inplace=True)
data.groupby(by='POI').compound.plot(legend=True, figsize=(12,8));
plt.savefig('Sentiment per location')

In [ ]:
data = df_tourist_en.groupby(by=['POI', pd.Grouper(key='created_at', freq='Y')]).compound.mean().reset_index()
data.set_index('created_at', inplace=True)
data.groupby(by='POI').compound.plot(legend=True, figsize=(15,8));

In [ ]:
data

In [ ]:
df_tourist_en['POI'].fillna(0, inplace=True)
df_tourist_en.loc[df_tourist_en['POI'] != 0]

In [ ]:
df_tourist_en.loc[df_tourist_en['POI'] == 'RAI']

In [ ]:
df_tourist_en['POI'].value_counts()

### Geospatial analysis

In [ ]:
df_tourist['type'].fillna(0, inplace=True)
gdf = geopandas.GeoDataFrame(
    df_tourist, geometry=geopandas.points_from_xy(df_tourist.longitude, df_tourist.latitude))

In [ ]:
gdf.sample(5)

In [ ]:
gdf['type'].head(5)

In [ ]:
gdf.crs = {'init' :'epsg:4326'}
gdf = gdf.to_crs(epsg=3857)

In [ ]:
gdf.info()

In [ ]:
gdf['latitude']=gdf['latitude'].astype('float')
gdf['longitude']=gdf['longitude'].astype('float')
gdf = gdf[gdf.latitude < 52.395] 
gdf = gdf[gdf.latitude > 52.34]
gdf = gdf[gdf.longitude < 4.94]
gdf = gdf[gdf.longitude > 4.85]

In [ ]:
%%capture
# %%capture ignores warnings
gdf['Longitude'] = gdf.geometry.apply(lambda pnt: pnt.x)
gdf['Latitude'] = gdf.geometry.apply(lambda pnt: pnt.y)

In [ ]:
ax = gdf.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')
contextily.add_basemap(ax, url=contextily.sources.ST_TERRAIN)

In [ ]:
def plot_map(data):
    """ Takes a GeoPandas.DataFrame as input """
    fig, ax = plt.subplots(1,1,figsize=(10,10))

    n_levels = 1000
    alpha_level = 0.5

    sns.kdeplot(
        data.Longitude, 
        data.Latitude,
        bw = 'scott',
        shade=True, 
        cmap='jet',
        kernel='gau',
        n_levels=n_levels,
        ax = ax,
        alpha=alpha_level,
        legend = True,
        shade_lowest = True
        #gridsize = 200
    )

    for i in range(len(ax.collections)-10):
        ax.collections[i].set_alpha(min(i/n_levels, alpha_level))

    contextily.add_basemap(ax, url=contextily.sources.ST_TERRAIN)
    
    #ax.set_ylim(6863000, 6869500)
    #ax.set_xlim(541000, 547500)

    #plt.axis('off')
    return ax

### Draw plot by selecting year (no arguments means all)

In [ ]:
gdf[gdf.created_at.dt.year == 2018].latitude.value_counts()

In [ ]:
gdf[gdf.created_at.dt.year == 2018].longitude.value_counts()

In [ ]:
df['longitude'] = df['longitude'].astype(str)
df['latitude'] = df['latitude'].astype(str)
df['latandlon'] = df['latitude'] +' '+ df['longitude']

In [ ]:
df['latandlon'].value_counts()

In [ ]:
df_checkbot = df[(df.latandlon =='52.3731 4.8922')]

In [ ]:
df_checkbot.head(5)

In [ ]:
df_checkbot.tail(5)

In [ ]:
gdf['longitude'] = gdf['longitude'].astype(str)
gdf['latitude'] = gdf['latitude'].astype(str)
gdf['latandlon'] = gdf['latitude'] +' '+ gdf['longitude']

In [ ]:
gdf['latandlon'].value_counts()

In [ ]:
gdf_checkbot = gdf[(gdf.latandlon =='52.3731 4.8922')]

In [ ]:
gdf['CoordinatesCount'] = gdf.groupby('latandlon')['latandlon'].transform('count')

In [ ]:
#Remove locations who have more than 1000 others with the same lat and lon coordinates
gdf = gdf[gdf.CoordinatesCount < 1000] 

In [ ]:
gdf.info()

In [ ]:
#Map of all years
ax = plot_map(gdf)
plt.axis('off')
plt.savefig('Heatmap all years before filtering')

In [ ]:
#Map per specific year
ax = plot_map(gdf[gdf.created_at.dt.year == 2016])
plt.axis('off')
plt.savefig('Heatmap 2016 before filtering')

### Select only attachments and plot gdf_attachments

In [ ]:
gdf['type'].value_counts()

In [ ]:
gdf_attachment = gdf.loc[gdf['type'] != 0]

In [ ]:
gdf_attachment.info()

In [ ]:
gdf_attachment['latandlon'].value_counts()

In [ ]:
gdf_attachment['type'].head(5)

In [ ]:
#Map with attachments
plot_map(gdf_attachment)#[gdf_attachment.created_at.dt.year == 2012])
plt.axis('off')
plt.savefig('Heatmap attachments')